In [1]:
from datasets import load_dataset 
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import time
import pandas as pd
from datasets import load_dataset 

dataset = load_dataset("NicolaiSivesind/human-vs-machine", "wiki_labeled")
dataset = dataset['test'].select(range(100))

/home/codespace/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 1. Load the model and tokeninzer

In [2]:
tokenizer = AutoTokenizer.from_pretrained("andreas122001/bloomz-3b-wiki-detector")
model = AutoModelForSequenceClassification.from_pretrained("andreas122001/bloomz-3b-wiki-detector")

Loading checkpoint shards: 100%|██████████| 2/2 [00:17<00:00,  8.98s/it]


### 2. Calculate the inference times for the given data

In [3]:
tokenized_texts = [tokenizer(x['text'], padding=True, truncation=True, return_tensors="pt", max_length=512).to("cpu") for x in dataset]
time_taken = []
model = model.to("cpu")

for text in tokenized_texts:
    start_time = time.time()
    with torch.no_grad():
        logits = model(**text).logits
    end_time = time.time()
    time_taken.append(end_time - start_time)
output = pd.DataFrame({"times":time_taken})
output.to_csv("base_inference_time.csv", index=False)
display(output.describe())

,times
count,100.000000
mean,22.190422
std,10.182044
min,6.119096
25%,14.109887
50%,20.726351
75%,29.180095
max,65.845625
